# **Minicurso: Introdução à Geologia De Sensoriamento Remoto com Python**

---

**XXXIV Semana de Estudos Geológicos da Universidade Federal Rural do Rio de Janeiro (SEGEO)**

### _Rodrigo Brust Santos_

#### Setembro 2024

---

**Tópicos:**
01) Introdução ao Sensoriamento Remoto & Python
02) Manipulando rasters
03) Índices de bandas, diferenças normalizadas, composições RGB
04) Sensoriamento Remoto Aplicado à Geologia

---

## 1 - Aplicação dos Conceitos em Regiões Específicas para Mineração


O sensoriamento remoto é uma ferramenta valiosa na exploração mineral, pois permite a identificação de padrões espectrais associados a diferentes tipos de minerais e formações geológicas. Em áreas de mineração, band ratios e índices como o Índice de Alteração de Ferro e o Índice de Óxidos de Ferro podem ser utilizados para identificar zonas com potencial de mineralização.

**Passo a passo:**

- Identificação de áreas de interesse: Usar dados de satélite de regiões conhecidas por atividades minerais.
- Band Ratios: Calcular ratios específicos que ajudam a identificar minerais comuns em processos de mineração (ex.: óxidos de ferro, minerais argilosos).
- Interpretação geológica: Analisar os resultados para identificar áreas com alta probabilidade de conter depósitos minerais.

In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
import gdrive_downloader

In [ ]:
#baixar imagem do gdrive

download_file_from_google_drive('https://drive.google.com/file/d/1LfhfVUgrjYw8AQzIhByNccJbqslvkXCg/view?usp=sharing',
                                os.getcwd() + '/Dados/geologia_libia.tif')

In [7]:
def normalize(image):
    return (image - np.min(image)) / (np.max(image) - np.min(image))

In [ ]:
# Carregar bandas relevantes (ex.: NIR e SWIR)

img_path = './Dados/L9_20240915_STACKED_Libia2.tif'

with rasterio.open(img_path) as src:
    blue = src.read(1)
    red = src.read(3)
    nir = src.read(5)
    swir = src.read(6)
    swir2 = src.read(7)

    #(R/B)*((R+S1)/N)

# Calcular um índice de óxido de ferro 
epsilon = 1e-10  # Evitar divisão por zero
iron_ferric = (red / blue) * ((red + swir) / (nir + epsilon))

# Plotar o índice
plt.imshow(iron_ferric, cmap='Reds_r')
plt.colorbar()
plt.title("Índice de Óxido de Ferro - Redness")
plt.show()


**Exercício**:

Use imagens de satélite de uma região de mineração e calcule o índice de alteração de ferro (ou outro índice relevante). Identifique visualmente áreas com potencial mineral.

In [ ]:
url = 'http://rodrigobrust.com/wp-content/uploads/2024/09/Rmount.tif'

#baixe a imagem conforme fizemos antes e faça outro índice

## 2 - Identificação de Diferenças Litológicas em Regiões Desérticas/Semi-desérticas

Em regiões desérticas e semi-desérticas, onde a vegetação é esparsa, o sensoriamento remoto pode ser usado para mapear diferenças litológicas com base nas respostas espectrais dos materiais da superfície. A análise de composições RGB permitem a distinção de diferentes formações rochosas e minerais de superfície.

**Passo a passo:**

- Selecionar bandas apropriadas: Em regiões áridas, as bandas do infravermelho médio (SWIR) e infravermelho próximo (NIR) são úteis para identificar argilas e outros minerais de superfície.
- Composição RGB: Criar composições de falsas cores para destacar diferenças litológicas.
- Aplicação de índices: Calcular índices que realçam minerais argilosos e alterações superficiais.

In [31]:
def stretch_histogram(band, min_out=0, max_out=255):
    min_in, max_in = np.percentile(band, (2, 98))  # Considerando 2% - 98% para evitar outliers
    stretched = (band - min_in) * ((max_out - min_out) / (max_in - min_in)) + min_out
    stretched = np.clip(stretched, min_out, max_out)
    return stretched

In [ ]:
# Carregar bandas SWIR e NIR
#with rasterio.open(img_path) as src:
#    swir = src.read(6)
#    nir = src.read(5)

# Composição RGB para diferenciação litológica
rgb_lithology = np.dstack((stretch_histogram(normalize(swir)), 
                           stretch_histogram(normalize(nir)), 
                           stretch_histogram(normalize(nir/2))
                           )
                        )

# Plotar a composição
plt.imshow(rgb_lithology / np.max(rgb_lithology))
plt.title("Composição RGB - Diferenças Litológicas")
plt.show()

**Exercício:**

Usando uma imagem de satélite acima, crie uma composição RGB para verificar se há constrastes entre rochas máficas/félsicas  na região. Nunca testei, será a primeira vez (:

[Veja aqui como se calcula o índice de Abrams](https://github.com/rodreras/awesome-mining-band-ratio?tab=readme-ov-file#rgb-composition-and-band-ratio-for-landsat-8-9-and-sentinel-2)

# Exercício para praticar depois: 

Abaixo, temos a url com as imagens raster de uma região na Mauritânia, próximo ao [trem mais longo do mundo](https://pt.wikipedia.org/wiki/Ferrovia_da_Maurit%C3%A2nia), com aproximadamente 3km, resopnsável por carregar Ferro até o litoral. 

O satélite é o Aster, por isso, precisam levar em consideração [esta tabela](https://github.com/rodreras/awesome-mining-band-ratio?tab=readme-ov-file#aster-band-ratios). Muito provavelmente, os índices que envolvam o NIR, Vermelho e SWIR (mais voltados para Ferro, por exemplo), apresentarão os melhores resultados, especialmente por conta do corpo maciço que há na região. Todavia, testem outros índices para ver como se comporta diferente em relação ao satélite que usamos anteriormente, que foi o Landasat (multiespectral)

In [ ]:
url_aster = ''